In [14]:
from oaklib import get_adapter
from oaklib.utilities.associations.edge_information_util import associations_to_matrix


ont_adapter = get_adapter("sqlite:obo:go")


In [47]:
taxa = ["9606", "6239", "7955", "3702", "559292", "4896", "7227", "10090", ]

In [48]:
matrix_by_taxon = {}
for tax in taxa:
    assoc_adapter = get_adapter(f"amigo:NCBITaxon:{tax}")
    assocs = list(assoc_adapter.associations(add_closure_fields=True))
    matrix = associations_to_matrix(assocs)
    matrix_by_taxon[tax] = matrix
    print(f"Taxon {tax} shape {matrix.arr.shape}")

Taxon 9606 shape (44398, 22647)
Taxon 6239 shape (14675, 9794)


ERROR:pysolr:Solr responded with an error (HTTP 503): [Reason: None]
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN"><html><head><title>503 Service Unavailable</title></head><body><h1>Service Unavailable</h1><p>The server is temporarily unable to service yourrequest due to maintenance downtime or capacityproblems. Please try again later.</p></body></html>


SolrError: Solr responded with an error (HTTP 503): [Reason: None]
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN"><html><head><title>503 Service Unavailable</title></head><body><h1>Service Unavailable</h1><p>The server is temporarily unable to service yourrequest due to maintenance downtime or capacityproblems. Please try again later.</p></body></html>

In [17]:
len(matrix_by_taxon)

2

132140

In [7]:
c="GO:0008527"
p="GO:0050912"


In [18]:
from oaklib.utilities.associations.edge_information_util import calculate_edge_information



In [19]:
for t, matrix in matrix_by_taxon.items():
    print(t, calculate_edge_information(c, p, matrix))

7227 0.12928301694496636
10090 0.405992359675837


In [38]:
rows = []
for s, p, o in ont_adapter.relationships():
    if not s.startswith("GO:"):
        continue
    if not o.startswith("GO:"):
        continue
    row = {"child": s, "parent": o, "predicate": p}
    for t, matrix in matrix_by_taxon.items():
        v = calculate_edge_information(s, o, matrix)
        row[t] = v
    rows.append(row)
    if len(rows) % 1000 == 0:
        print(row)
        

{'child': 'GO:0000943', 'parent': 'GO:0110165', 'predicate': 'rdfs:subClassOf', '7227': None, '10090': None}
{'child': 'GO:0001933', 'parent': 'GO:0006468', 'predicate': 'RO:0002212', '7227': 2.8187596853298533, '10090': 1.0951572330403403}
{'child': 'GO:0002503', 'parent': 'GO:0002495', 'predicate': 'BFO:0000050', '7227': None, '10090': 0.8073549220576037}
{'child': 'GO:0002869', 'parent': 'GO:0002904', 'predicate': 'rdfs:subClassOf', '7227': None, '10090': None}
{'child': 'GO:0003678', 'parent': 'GO:0032508', 'predicate': 'BFO:0000050', '7227': 0.22239242133644765, '10090': 0.32192809488736174}
{'child': 'GO:0004823', 'parent': 'GO:0004812', 'predicate': 'rdfs:subClassOf', '7227': 4.247927513443585, '10090': 4.357552004618084}
{'child': 'GO:0005743', 'parent': 'GO:0019866', 'predicate': 'rdfs:subClassOf', '7227': 0.05916207246814409, '10090': 0.09780478547179872}
{'child': 'GO:0006364', 'parent': 'GO:0042254', 'predicate': 'BFO:0000050', '7227': 0.5341133592254037, '10090': 0.5756846

In [39]:
import pandas as pd


In [40]:
def has_val(row: dict):
    for k, v in row.items():
        if k not in ["child", "parent", "predicate"]:
            if v is not None:
                return True
    return False

In [41]:
filtered_rows = [r for r in rows if has_val(r)]

In [42]:
for row in filtered_rows:
    row["child_label"] = ont_adapter.label(row["child"])
    row["parent_label"] = ont_adapter.label(row["parent"])

In [43]:
df = pd.DataFrame(filtered_rows)
df

,child,parent,predicate,7227,10090,child_label,parent_label
0,GO:0000001,GO:0005739,obo:GOREL_0002003,7.813781,NaN,mitochondrion inheritance,mitochondrion
1,GO:0000001,GO:0048308,rdfs:subClassOf,0.000000,NaN,mitochondrion inheritance,organelle inheritance
2,GO:0000001,GO:0048311,rdfs:subClassOf,1.459432,NaN,mitochondrion inheritance,mitochondrion distribution
3,GO:0000002,GO:0007005,rdfs:subClassOf,4.650254,3.865499,mitochondrial genome maintenance,mitochondrion organization
4,GO:0000009,GO:0000030,rdfs:subClassOf,3.321928,3.643856,"alpha-1,6-mannosyltransferase activity",mannosyltransferase activity
...,...,...,...,...,...,...,...
49354,GO:2001306,GO:1901617,rdfs:subClassOf,NaN,7.491853,lipoxin B4 biosynthetic process,organic hydroxy compound biosynthetic process
49355,GO:2001306,GO:2001301,rdfs:subClassOf,NaN,2.000000,lipoxin B4 biosynthetic process,lipoxin biosynthetic process
49356,GO:2001306,GO:2001304,rdfs:subClassOf,NaN,0.000000,lipoxin B4 biosynthetic process,lipoxin B4 metabolic process
49357,GO:2001311,GO:0006650,rdfs:subClassOf,NaN,7.049849,lysobisphosphatidic acid metabolic process,glycerophospholipid metabolic process


In [44]:
!mkdir results

mkdir: results: File exists


In [45]:
df.to_csv("results/edge_information.csv", index=False)

In [46]:
old_matrix_by_taxon = matrix_by_taxon